<a href="https://colab.research.google.com/github/ykitaguchi77/GO_AI_project/blob/main/make_super_extend_dataset_for_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Create extended dataset for YOLOv5 implementation**

#Olympia
・Olympia dataset

Treated 640px: 561枚。テストに50枚を残す（test_olympia）、残りの511枚をtrain/valに回す

Control 640px: 169枚。テストに50枚を残す(test_control)、残りの119枚をtrain/valに回す

#Handai
・GO_extended_dataset

control: 1886枚。テストに50枚を残す(test_handai)、残りの1836枚をtrain/valに回す

treatable: 1879枚。テストに50枚を残す(test_handai)、残りの1829枚をtrain/valに回す


#合計：
treatable: 1829 + 119

control: 1836 + 511

testset: Olympia treated 50, control 50, Handai treated 50, control 50

##既存のYOLOでbounding boxを作成 --> classは既知のもので。Glaucomaのrepositoryを参考にしてYOLO datasetを作成。

In [17]:
import pandas as pd
import os
import glob
import shutil
import glob
import random
from PIL import Image
import time
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2


# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#ファイル数確認用
import os

def countfile(name, path):
    print(f"path: {path}")
    print(f"number: {len(os.listdir(path))}")
    print("")

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/


olympia_cont_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/control_640px"
olympia_grav_dir = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px"
handai_cont_dir = "/content/GO_extended_dataset/Control_photo_1886mai"
handai_grav_dir = "/content/GO_extended_dataset/treatable"


countfile("control_640px_olympia", olympia_cont_dir)
countfile("treated_640px_olympia", olympia_grav_dir)

countfile("control_handai", handai_cont_dir)
countfile("treated_handai", handai_grav_dir)


path: /content/drive/MyDrive/Deep_learning/Olympia_dataset/control_640px
number: 169

path: /content/drive/MyDrive/Deep_learning/Olympia_dataset/treated_640px
number: 561

path: /content/GO_extended_dataset/Control_photo_1886mai
number: 1886

path: /content/GO_extended_dataset/treatable
number: 1879



#**Handai dataset, Olympia dataset --> super_extend_datase_for_YOLO**

In [21]:
dst_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)
os.makedirs(f"{dst_dir}/train_val")
os.makedirs(f"{dst_dir}/train_val/grav")
os.makedirs(f"{dst_dir}/train_val/cont")
os.makedirs(f"{dst_dir}/test")
os.makedirs(f"{dst_dir}/test/handai/grav")
os.makedirs(f"{dst_dir}/test/handai/cont")
os.makedirs(f"{dst_dir}/test/olympia/grav")
os.makedirs(f"{dst_dir}/test/olympia/cont")

In [22]:
import os
import shutil
import random
from tqdm import tqdm

random_state = 1

# Control images
src_cont = olympia_cont_dir
test_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
trainval_cont = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

cont_images = os.listdir(src_cont)
random.Random(random_state).shuffle(cont_images)

test_cont_images = cont_images[:50]
trainval_cont_images = cont_images[50:]

for image in tqdm(test_cont_images, desc="Copying test cont images"):
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(test_cont, image)
    shutil.copy(src_path, dst_path)

for image in tqdm(trainval_cont_images, desc="Copying trainval cont images"):
    src_path = os.path.join(src_cont, image)
    dst_path = os.path.join(trainval_cont, image)
    shutil.copy(src_path, dst_path)


# Grav images
src_grav = olympia_grav_dir
test_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
trainval_grav = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"

grav_images = os.listdir(src_grav)
random.Random(random_state).shuffle(grav_images)

test_grav_images = grav_images[:50]
trainval_grav_images = grav_images[50:]

for image in tqdm(test_grav_images, desc="Copying test grav images"):
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(test_grav, image)
    shutil.copy(src_path, dst_path)

for image in tqdm(trainval_grav_images, desc="Copying trainval grav images"):
    src_path = os.path.join(src_grav, image)
    dst_path = os.path.join(trainval_grav, image)
    shutil.copy(src_path, dst_path)





Copying trainval grav images: 100%|██████████| 511/511 [00:10<00:00, 47.54it/s]


In [23]:
import os
from shutil import copyfile
from tqdm import tqdm

grav_dir = olympia_grav_dir
cont_dir = olympia_cont_dir
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# Check if files in grav_dir are in test_grav_dir
for filename in tqdm(os.listdir(grav_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_grav_dir, basename)):
        print("coooool!!!")
        continue
    else:
        copyfile(os.path.join(grav_dir, filename), os.path.join(train_val_grav_dir, filename))

# Check if files in cont_dir are in test_cont_dir
for filename in tqdm(os.listdir(cont_dir)):
    basename = os.path.basename(filename)
    if os.path.exists(os.path.join(test_cont_dir, basename)):
        print("hotttttt!!!")
        continue
    else:
        copyfile(os.path.join(cont_dir, filename), os.path.join(train_val_cont_dir, filename))


  3%|▎         | 15/561 [00:00<00:08, 63.11it/s]

coooool!!!


 10%|▉         | 56/561 [00:00<00:08, 60.29it/s]

coooool!!!
coooool!!!


 14%|█▍        | 79/561 [00:01<00:08, 59.11it/s]

...

hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!


 86%|████████▌ | 145/169 [00:01<00:00, 83.71it/s]

hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!


100%|██████████| 169/169 [00:02<00:00, 77.93it/s]

hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!
hotttttt!!!


In [24]:
#####################
# rename
#指定されたフォルダ内にあるファイル名が"olympia_"から始まらない場合に、"olympia_"を先頭につけたファイル名にリネームする処理を行う。
#####################

import os

# フォルダのパスを指定
test_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/grav"
test_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/olympia/cont"
train_val_grav_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav"
train_val_cont_dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont"

# ファイル名が "olympia_" から始まらない場合、"olympia_" を先頭につけたファイル名にリネームする
def rename_file(path):
    for filename in os.listdir(path):
        if not filename.startswith("oly_"):
            new_filename = "oly_" + filename
            os.rename(os.path.join(path, filename), os.path.join(path, new_filename))

# リネーム処理を実行
rename_file(test_grav_dir)
rename_file(test_cont_dir)
rename_file(train_val_grav_dir)
rename_file(train_val_cont_dir)


In [26]:
print(len(os.listdir("/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav")))
print(len(os.listdir("/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont")))


511
119


In [50]:
from sklearn.model_selection import StratifiedShuffleSplit
import os
import random
import shutil
from tqdm import tqdm

############################
# Handai GO_extended_dataset
#
# /treatable → /train_val/grav
# /control → /train_val/Control_photo_1886mai
#
############################

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/

#file_name, class, idxのりストを作成
def get_file_info(src_dir, class_id):
    file_list = os.listdir(src_dir)
    ids_list = [int(file_name.split('.')[0].split('-')[0]) for file_name in file_list]
    class_list = [class_id] * len(file_list)
    return file_list, ids_list, class_list

# Contフォルダの情報
cont_dir = '/content/GO_extended_dataset/Control_photo_1886mai'
file_list_cont, ids_list_cont, class_list_cont = get_file_info(cont_dir, 0)

# Gravフォルダの情報
grav_dir = '/content/GO_extended_dataset/treatable'
file_list_grav, ids_list_grav, class_list_grav = get_file_info(grav_dir, 1)

# 結合
file_list = file_list_cont + file_list_grav
ids_list = ids_list_cont + ids_list_grav
class_list = class_list_cont + class_list_grav

print(file_list)
print(ids_list)
print(class_list)


train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/cont'

train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/grav'

# # ファイルリストを取得
# file_list_grav = os.listdir(src_dir)
# print(file_list_grav)

# # IDを取得
# ids_list_grav = [int(file_name.split('.')[0].split('-')[0]) for file_name in file_list_grav]
# print(ids_list_grav)

# class_list_grav = [1]*len(file_list_grav)
# print(class_list_grav)


# file_list = file_list_cont + file_list_grav
# ids_list = ids_list_cont + ids_list_grav
# class_list = class_list_cont + class_list_grav


# Contのgravに対する割合を計算
cont_grav_ratio = sum([1 for file_name in file_list if "grav" in file_name]) / len(file_list)

# StratifiedShuffleSplitによるデータ分割
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=2)
trainval_indices, test_indices = next(sss.split(file_list, ids))

# テスト用のフォルダにコピー
for i in test_indices:
    file_name = file_list[i]
    src_path = os.path.join(src_dir, file_name)
    dst_path = os.path.join(test_dir, file_name)
    shutil.copy(src_path, dst_path)

# 学習用のフォルダにコピー
for i in trainval_indices:
    file_name = file_list[i]
    src_path = os.path.join(src_dir, file_name)
    dst_path = os.path.join(train_val_dir, file_name)
    shutil.copy(src_path, dst_path)

# Gravフォルダのパス
src_dir = '/content/GO_extended_dataset/treatable'
train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/grav'

# ファイルリストを取得
file_list = os.listdir(src_dir)

# IDを取得
ids = [int(file_name.split('-')[0]) for file_name in file_list]

# Gravのcontに対する割合を計算
grav_cont_ratio = sum([1 for file_name in file_list if "cont" in file_name]) / len(file_list)

# Contのgravに対する割合とGravのcontに対する割合が同じになるようにStratifiedShuffleSplitによるデータ分割
while True:
    sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=2)
    trainval_indices, test_indices = next(sss.split(file_list, ids))

    # Contのgravに対する割合とGravのcontに対する割合を計算
    cont_grav_ratio_trainval = sum([1 for i in trainval_indices if "grav" in file_list[i]]) / len(trainval_indices)
    grav_cont_ratio_trainval = sum([1 for i in trainval_indices if "cont" in file_list[i]]) / len(trainval_indices)

    # Contのgravに対する割合とGravのcontに対する割合が同じであればデータ分割処理を終了
    if abs(cont_grav_ratio - cont_grav_ratio_trainval) < 0.01 and abs(grav_cont_ratio - grav_cont_ratio_trainval) < 0.01:
        break

# テスト用のフォルダにコピー
for i in test_indices:
    file_name = file_list[i]
    src_path = os.path.join(src_dir, file_name)
    dst_path = os.path.join(test_dir, file_name)
    shutil.copy(src_path, dst_path)

# 学習用のフォルダにコピー
for i in trainval_indices:
    file_name = file_list[i]
    src_path = os.path.join(src_dir, file_name)
    dst_path = os.path.join(train_val_dir, file_name)
    shutil.copy(src_path, dst_path)

['9518-20210623-62-135900_332522d93c135b334f82f5462170de29fafc9f54bcb2b077ae2f609f53348b78.jpg', '9505-20210506-47-144526_4b91547f27ca8847805ab34e7cc3114beb25fb57e03668d715501ba97c94316f.jpg', '6523.jpg', '9498-20201014-60-135741_bb57faf0ad1bf1cde291ffd69c8ab0ac29704c6b74a662c29acc68c649e9de1b.jpg', '9168-20210324-34-110132_c6f214537e246b4fa049335e5f8c751fff3a267c4c2b371d2a643039ab55fd34.jpg', '4374.jpg', '2567.jpg', '3305.jpg', '6720.jpg', '9293-20200729-3-111234_e71fc2814a25e048776a8443d77b1cd52af17fdf15d6b5dd654ab1f48a6bcc97.jpg', '1655.jpg', '2193.JPG', '4065.jpg', '3939.jpg', '3773.jpg', '1690.JPG', '5082.JPG', '735.jpg', '9378-20210219-38-111426_f9b3a28684240ed221441e1f7e95f5fb09092939cfd0fbaa1bcd34fd1632ab35.jpg', '6081.jpg', '2429.jpg', '5981.jpg', '6465.jpg', '4098.jpg', '2866.jpg', '7514.jpg', '4525.JPG', '891.jpg', '7181.JPG', '137.jpg', '1646.jpg', '1424.JPG', '7632.JPG', '2145.jpg', '806.jpg', '2563.JPG', '9688-20191219-25-145751_556e0bc1a30322d6aca440dc3eef9ef52cf654c5e25

ValueError: ignored

In [ ]:
import os
import random
import shutil
from tqdm import tqdm

############################
# Handai GO_extended_dataset
#
# /treatable → /train_val/grav
# /control → /train_val/Control_photo_1886mai
#
############################

if not os.path.exists("/content/GO_extended_dataset"):
    !unzip /content/drive/MyDrive/Deep_learning/GO_extended_dataset/GO_extended_dataset.zip -d /content/

# Contフォルダのパス
src_dir = '/content/GO_extended_dataset/Control_photo_1886mai'
train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/cont'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/cont'

# ファイルリストを取得し、ランダムにシャッフル
file_list = os.listdir(src_dir)
random.Random(2).shuffle(file_list)

# テスト用のフォルダに50枚コピーし、残りを学習用のフォルダにコピー
for i, file_name in tqdm(enumerate(file_list), total=len(file_list)):
    src_path = os.path.join(src_dir, file_name)
    dst_dir = test_dir if i < 50 else train_val_dir
    dst_path = os.path.join(dst_dir, file_name)
    shutil.copy(src_path, dst_path)

# Gravフォルダのパス
src_dir = '/content/GO_extended_dataset/treatable'
train_val_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/train_val/grav'
test_dir = '/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO/test/handai/grav'

# ファイルリストを取得し、ランダムにシャッフル
file_list = os.listdir(src_dir)
random.Random(2).shuffle(file_list)

# テスト用のフォルダに50枚コピーし、残りを学習用のフォルダにコピー
for i, file_name in tqdm(enumerate(file_list), total=len(file_list)):
    src_path = os.path.join(src_dir, file_name)
    dst_dir = test_dir if i < 50 else train_val_dir
    dst_path = os.path.join(dst_dir, file_name)
    shutil.copy(src_path, dst_path)

100%|██████████| 1879/1879 [00:37<00:00, 50.57it/s]


In [ ]:
"""
ToDo：
train_val, testで分けたが、はじめからtrain/val/testで分けてもよいような気がする
GravについてはStratified Group foldで分ける必要がある (train, val, test)

"""

#**ファイル数の確認**

In [ ]:
dir = "/content/drive/MyDrive/Deep_learning/GO_extended_dataset/super_extend_dataset_for_YOLO"
print(len(os.listdir(f"{dir}/train_val/grav")))
print(len(os.listdir(f"{dir}/train_val/cont")))
print(len(os.listdir(f"{dir}/test/olympia/grav")))
print(len(os.listdir(f"{dir}/test/olympia/cont")))
print(len(os.listdir(f"{dir}/test/handai/grav")))
print(len(os.listdir(f"{dir}/test/handai/cont")))

2340
1955
50
50
50
50
